In [1]:
import pandas as pd
from functools import reduce
import json

In [2]:
s = "{\"pk\":2,\"coefficient\":[],\"variable\":[{\"key\":\"P\",\"val\":[\"excel/99.xls\",\"Sheet1\",\"1段QUALITY\"]},{\"key\":\"A\",\"val\":[\"excel/sc.csv\",\"_csv_\",\"工厂\"]},{\"key\":\"T\",\"val\":[\"excel/99.xls\",\"Sheet1\",\"1段PRICE\"]}]}"
l = json.loads(s)
path = '/mnt/code/djcode/teserver/media/'
variables=l['variable']

In [3]:
readDict = {}
valuesDict = {}
for variable in variables:
    k = None
    if variable['val'][1] == '_csv_':
        k = variable['val'][0]
        if k not in readDict:
            real_path = path+k
            readDict[k] = pd.read_csv(real_path)
    else:
        k = variable['val'][0]+'-*-'+variable['val'][1]
        if k not in readDict:
            readDict[k] = pd.read_excel(
                path+variable['val'][0], sheet_name=variable['val'][1])
    valuesDict[variable['key']] = {}
    valuesDict[variable['key']]['key'] = variable['val'][2]
    valuesDict[variable['key']]['reader'] = readDict[k]

In [4]:
valuesDict

{'P': {'key': '1段QUALITY',
  'reader':      1段PRICE  1段QUALITY  2段PRICE  2段QUALITY  3段PRICE  3段QUALITY  4段PRICE  \
  0    11.7472    17132.2   8.9972    16271.1   6.7942    15809.6   5.9607   
  1    14.9908    16404.4  12.8299    15693.9  11.7459    15368.1  11.6355   
  2    14.9954    16470.3  14.9939    16067.9  13.8483    15673.7  12.7867   
  3    15.9952    16515.4  14.8973    16162.5  14.8424    15938.3  12.9922   
  4    15.9966    16958.1  15.9900    16314.9  11.9983    16069.6  12.9957   
  ..       ...        ...      ...        ...      ...        ...      ...   
  362  25.4950    18534.9  24.5083    18174.3  24.0083    17875.7  24.5014   
  363  27.4727    19373.2  25.4925    18652.0  25.0024    18483.5  25.0004   
  364  27.9971    19302.4  22.9962    18544.2  22.2552    18157.9  21.9274   
  365      NaN        NaN      NaN        NaN      NaN        NaN      NaN   
  366  65.7923    23092.1  51.6830    21930.3  41.2713    21478.0  40.8822   
  
       4段QUALITY  5段PRIC

In [5]:
p = valuesDict['P']['reader'][valuesDict['P']['key']]
a = valuesDict['A']['reader'][valuesDict['A']['key']]
t = valuesDict['T']['reader'][valuesDict['T']['key']]


In [6]:
import numpy as np
pp = np.int32(p)

In [7]:
pp

array([      17132,       16404,       16470,       16515,       16958,
             17593,       17775,       18252,       18101,       17181,
             16901,       17355,       17908,       18339,       17787,
             17474,       16389,       16249,       17168,       17332,
             16641,       15947,       16435,       16285,       16576,
             16816,       17105,       17137,       17394,       17292,
             16421,       16427,       17322,       17196,       17375,
             16358,       16287,       15343,       15310,       16090,
             16584,       16663,       16861,       16634,       15578,
             15703,       16166,       16000,       16687,       16034,
             16172,       15876,       15428,       16197,       16401,
             16121,       16163,       16582,       16611,       15606,
             16528,       16751,       16860,       16483,       16584,
             16152,       15747,       16665,       17314,      

In [8]:
# 1.method = 'ffill'/'pad'：用前一个非缺失值去填充该缺失值
# 2.method = 'bflii'/'backfill'：用下一个非缺失值填充该缺失值
# 指定axis参数 df2.fillna(method="ffill", limit=1, axis=1)

aa = pd.to_numeric(a, errors='coerce').fillna(method='ffill')

In [9]:
r = p*aa*t

In [10]:
aa.fillna(method='ffill')

0      9800.0
1      9010.0
2      9800.0
3      9800.0
4      9800.0
        ...  
648    9010.0
649    9010.0
650    9800.0
651    9800.0
652    9800.0
Name: 工厂, Length: 653, dtype: float64

In [11]:
r.dropna()

0      1.972303e+09
1      2.215695e+09
2      2.420392e+09
3      2.588838e+09
4      2.658465e+09
           ...     
361    4.616956e+09
362    4.630963e+09
363    5.215894e+09
364    5.296030e+09
366    1.488897e+10
Length: 366, dtype: float64

In [12]:
tmp = []
for i in valuesDict.keys():
    value = valuesDict[i]
    v = value['reader'][value['key']]
    if v.dtype.name == 'object':
        v = pd.to_numeric(v,errors='coerce')
    tmp.append(v.fillna(method='ffill'))

In [13]:
tmp

[0      17132.2
 1      16404.4
 2      16470.3
 3      16515.4
 4      16958.1
         ...   
 362    18534.9
 363    19373.2
 364    19302.4
 365    19302.4
 366    23092.1
 Name: 1段QUALITY, Length: 367, dtype: float64,
 0      9800.0
 1      9010.0
 2      9800.0
 3      9800.0
 4      9800.0
         ...  
 648    9010.0
 649    9010.0
 650    9800.0
 651    9800.0
 652    9800.0
 Name: 工厂, Length: 653, dtype: float64,
 0      11.7472
 1      14.9908
 2      14.9954
 3      15.9952
 4      15.9966
         ...   
 362    25.4950
 363    27.4727
 364    27.9971
 365    27.9971
 366    65.7923
 Name: 1段PRICE, Length: 367, dtype: float64]

In [14]:
r = reduce(lambda x, y: x*y,tmp)

In [22]:
r = r.dropna()

In [26]:
rrr = r.tolist()

In [46]:
x = list(zip(range(len(rrr)),rrr))
x

[(0, 1972302722.432),
 (1, 2215694866.4752),
 (2, 2420391618.876),
 (3, 2588837835.584),
 (4, 2658465036.1080003),
 (5, 2929407024.866),
 (6, 3443825548.94),
 (7, 3575928678.896),
 (8, 3380527616.0568),
 (9, 3396377735.562),
 (10, 3168738697.32),
 (11, 3157280892.8640003),
 (12, 3088734014.058),
 (13, 3256618243.7879996),
 (14, 3312642611.25),
 (15, 3457930338.609),
 (16, 3416213285.03),
 (17, 3026429099.232),
 (18, 2919529486.1879997),
 (19, 1978897193.0),
 (20, 1793513365.9099998),
 (21, 1061919745.656),
 (22, 1702619290.5856001),
 (23, 2076122858.838),
 (24, 1136607490.368),
 (25, 2119824205.0155),
 (26, 2345303393.146),
 (27, 2266789930.314),
 (28, 2350873716.5853),
 (29, 2286607760.892),
 (30, 2293140766.389),
 (31, 1930492546.5279999),
 (32, 2106550613.4594),
 (33, 2052642901.2903001),
 (34, 2506349848.1),
 (35, 2062480068.7552001),
 (36, 2233454832.9240003),
 (37, 1762883011.68),
 (38, 1654859002.3966),
 (39, 2028575914.2129002),
 (40, 2274770719.76),
 (41, 2285274581.436),
 (42

In [47]:
npx = np.array(x)

In [52]:
xxxx = npx.astype(str)

array([['0.0', '1972302722.432'],
       ['1.0', '2215694866.4752'],
       ['2.0', '2420391618.876'],
       ['3.0', '2588837835.584'],
       ['4.0', '2658465036.1080003'],
       ['5.0', '2929407024.866'],
       ['6.0', '3443825548.94'],
       ['7.0', '3575928678.896'],
       ['8.0', '3380527616.0568'],
       ['9.0', '3396377735.562'],
       ['10.0', '3168738697.32'],
       ['11.0', '3157280892.8640003'],
       ['12.0', '3088734014.058'],
       ['13.0', '3256618243.7879996'],
       ['14.0', '3312642611.25'],
       ['15.0', '3457930338.609'],
       ['16.0', '3416213285.03'],
       ['17.0', '3026429099.232'],
       ['18.0', '2919529486.1879997'],
       ['19.0', '1978897193.0'],
       ['20.0', '1793513365.9099998'],
       ['21.0', '1061919745.656'],
       ['22.0', '1702619290.5856001'],
       ['23.0', '2076122858.838'],
       ['24.0', '1136607490.368'],
       ['25.0', '2119824205.0155'],
       ['26.0', '2345303393.146'],
       ['27.0', '2266789930.314'],
       ['

In [ ]:
rr = r.astype(str)
"_|_".join(rr)
# "_|_".join(r.values.tolist())

'1972302722.432_|_2215694866.4752_|_2420391618.876_|_2588837835.584_|_2658465036.1080003_|_2929407024.866_|_3443825548.94_|_3575928678.896_|_3380527616.0568_|_3396377735.562_|_3168738697.32_|_3157280892.8640003_|_3088734014.058_|_3256618243.7879996_|_3312642611.25_|_3457930338.609_|_3416213285.03_|_3026429099.232_|_2919529486.1879997_|_1978897193.0_|_1793513365.9099998_|_1061919745.656_|_1702619290.5856001_|_2076122858.838_|_1136607490.368_|_2119824205.0155_|_2345303393.146_|_2266789930.314_|_2350873716.5853_|_2286607760.892_|_2293140766.389_|_1930492546.5279999_|_2106550613.4594_|_2052642901.2903001_|_2506349848.1_|_2062480068.7552001_|_2233454832.9240003_|_1762883011.68_|_1654859002.3966_|_2028575914.2129002_|_2274770719.76_|_2285274581.436_|_2395532993.952_|_2246998623.2935_|_2235953335.28_|_1979676199.1692_|_1980880207.614_|_1802948308.3565001_|_1961919519.826_|_1957769570.372_|_2217814555.264_|_1988184006.8860002_|_1813366811.256_|_2221489607.4_|_2142698364.9503999_|_2202546211.71